In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/train-en-ms.tar.gz
# !tar -zxf train-en-ms.tar.gz
# !rm train-en-ms.tar.gz

In [2]:
# !pip3 install pyenchant

In [3]:
# import enchant
# d = enchant.Dict("en_US")
# d.check("Hello")

In [4]:
# !wget https://raw.githubusercontent.com/huseinzol05/malay-dataset/master/normalization/en-lexicon/en-social-media-lexicon.json
# !wget https://raw.githubusercontent.com/huseinzol05/malay-dataset/master/normalization/en-lexicon/en-social-media-lexicon-v2.json

In [5]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [6]:
!ls -lha train-en

total 1.6G
drwxr-xr-x  2 ubuntu ubuntu 4.0K Jun  28  2020 .
drwxr-xr-x 45 ubuntu ubuntu  12K Jul  13 18:58 ..
-rw-r--r--  1 ubuntu ubuntu 772M Ogos  2  2020 left.txt
-rw-r--r--  1 ubuntu ubuntu 860M Ogos  2  2020 right.txt


In [7]:
with open('train-en/left.txt') as fopen:
    left = fopen.read().split('\n')
    
with open('train-en/right.txt') as fopen:
    right = fopen.read().split('\n')

In [8]:
left[7106], right[7106]

('The goal is to make sure the people who receive the land are the ones who really deserve it.',
 'Tujuannya bagi memastikan pihak yang menerima tanah itu adalah mereka yang benar-benar layak.')

In [9]:
len(left), len(right)

(3807616, 3807616)

In [10]:
import malaya

/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.6.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops again

In [11]:
from malaya.text.rules import rules_normalizer, rules_compound_normalizer
from malaya.text.normalization import _is_number_regex
from collections import defaultdict
import re
import random
import json

In [12]:
with open('en-social-media-lexicon.json') as fopen:
    en_lexicon = json.load(fopen)
    
with open('en-social-media-lexicon-v2.json') as fopen:
    en_lexicon_v2 = json.load(fopen)

In [13]:
compound_normalizer = defaultdict(list)
normalizer = defaultdict(list)

for k, v in en_lexicon.items():
    if not len(v):
        continue
    if len(k.split()) > 1:
        compound_normalizer[k].extend(v)
    else:
        normalizer[k].extend(v)

for k, v in en_lexicon_v2.items():
    if not len(v):
        continue
    if len(k.split()) > 1:
        compound_normalizer[k].extend(v)
    else:
        normalizer[k].extend(v)
        
compound_normalizer_regex = (
    '(?:' + '|'.join(list(compound_normalizer.keys())) + ')'
)

In [14]:
PUNCTUATION = '!"#$%&\'()*+,./:;<=>?@[\]^_`{|}~'

def case_of(text):
    return (
        str.upper
        if text.isupper()
        else str.lower
        if text.islower()
        else str.title
        if text.istitle()
        else str
    )

def strip_punct(word):
    left = []
    right = []
    i = 0
    while i < len(word):
        if word[i] in PUNCTUATION:
            left.append(word[i])
            i += 1
        else:
            break
    i = len(word) - 1
    while i > 0:
        if word[i] in PUNCTUATION:
            right.append(word[i])
            i -= 1
        else:
            break
    left = ''.join(left)
    right = ''.join(right[::-1])
    if len(right):
        word_ = word[:-len(right)]
    else:
        word_ = word
    word_ = word_[len(left):]
    return left, right, word_


def replace_shortword(word, rules = normalizer):
    left, right, word_ = strip_punct(word)
    word_ = word_[len(left):]
    lower_word_ = word_.lower()
    if lower_word_ in rules:
        word_ = case_of(word_)(random.choice(rules[lower_word_]))
        word_ = f'{left}{word_}{right}'
        return word_
    else:
        return word
    
strip_punct('counters,')

('', ',', 'counters')

In [15]:
# def _replace_compound(string, 
#                       rules_regex = rules_compound_normalizer_regex, 
#                       rules = rev_rules_compound_normalizer):
#     results = re.findall(rules_regex, string, flags=re.IGNORECASE
#     )
#     for r in results:
#         try:
#             string = string.replace(r, random.choice(rules[r.lower()]))
#         except:
#             pass
#     return string

def _replace_compound(string, rules = compound_normalizer):
    for k in list(rules.keys()):
        results = [(m.start(0), m.end(0)) for m in re.finditer(k, string, flags=re.IGNORECASE)]
        for r in results:
            sub = string[r[0]: r[1]]
            try:
                replaced = random.choice(rules[sub.lower()])
                if replaced:
                    if r[1] < len(string) and string[r[1]] != ' ':
                        continue
                    if r[0] - 1 > len(string) and string[r[0] - 1] != ' ':
                        continue

                    sub = case_of(sub)(replaced)
                    string = string[:r[0]] + sub + string[r[1]:]
            except:
                pass
    return string

In [16]:
_replace_compound('i like, text to cell', rules = compound_normalizer)

'i like, ttc'

In [17]:
string = 'i am so beautiful"'
[replace_shortword(word) for word in string.split()]

['im', 'ham', 'soooooooooo', 'beutifall"']

In [18]:
[replace_shortword(word) for word in left[1].split()]

['Pitt',
 'his',
 'senss',
 'addopted*2',
 'buth',
 'cheon.',
 'Ir',
 '2006,',
 'th*2',
 'coup',
 'hab',
 "it's*2",
 'firts',
 'biological',
 'dauter,',
 'Shiloh',
 'Nouvel,',
 'want',
 'prefers',
 'taa',
 'linge',
 'kalled',
 'Johhn,',
 'acording*2',
 'taa',
 'Tu',
 'Telagraph.',
 'Thoy',
 'cupel',
 'addopted*2',
 'Pax',
 'Thien,',
 'nme',
 '12,',
 'befrou',
 'Jolie',
 'gav',
 'berth',
 'tow*2',
 'twins',
 'Knox',
 'Leon',
 'i',
 'Vivienne',
 'Marcheline,',
 'nuh',
 'seven.',
 'Air',
 'Januari,',
 'Jolie',
 'madh',
 'headlines',
 'foooooor',
 'bringin',
 'them',
 'kidz',
 'tooo',
 'thoy',
 'premeire',
 'ove',
 '"Kung',
 'Fu',
 'Panda',
 '3".',
 'Jolie',
 'e',
 'Pitt',
 'olso',
 'reportedly',
 'berng',
 'alongggg',
 'aa',
 'tem',
 'afoh',
 'nannies',
 'e',
 'tearch',
 'were_ever',
 'thror',
 'travle.',
 'Acording*2',
 'tp',
 'Jolie,',
 'million_ear',
 'moms',
 'shoudln',
 'complane',
 'abuot',
 'juggling',
 'wrk',
 'i',
 'famliy',
 'bc',
 'thar',
 'hav',
 'mooe',
 'acess,',
 'suppart',

In [19]:
def replace_words_punct(left_word, right_word):
    left_left, left_right, left_word = strip_punct(left_word)
    right_left, right_right, right_word = strip_punct(right_word)
    return f'{left_left}{right_word}{left_right}'

def random_replace_alignment(left, right, alignment, min_replace = 5, max_replace = 10):
    splitted_left = left.split()
    splitted_right = right.split()
    
    selected_alignment = []
    for s in alignment:
        l = s[0]
        r = s[1]
        try:
            if _is_number_regex(splitted_left[l].replace(',', '').replace('.', '')) or _is_number_regex(splitted_right[r].replace(',', '').replace('.', '')):
                continue
            elif splitted_left[l].isupper() or splitted_right[r].isupper():
                continue
            elif splitted_left[l] == splitted_right[r]:
                continue
            elif splitted_left[r].lower() in ['the', 'a', 'an', 'it', 'is', 'are']:
                continue
            else:
                selected_alignment.append((l, r))
        except:
            pass
    
    try:
        count_replace = random.randint(min_replace, min(max_replace, len(selected_alignment)))
        selected = random.sample(selected_alignment, count_replace)
        for s in selected:
            splitted_left[s[0]] = replace_words_punct(splitted_left[s[0]], splitted_right[s[1]])

        return ' '.join(splitted_left)
    
    except:
        return ' '.join(splitted_left)

In [20]:
eflomal = malaya.alignment.en_ms.eflomal()

In [21]:
%%time

alignment = eflomal.align(left[7106:7107], right[7106:7107])['forward'][0]

CPU times: user 3.96 ms, sys: 32 ms, total: 36 ms
Wall time: 172 ms


In [22]:
random_replace_alignment(left[7106], right[7106], alignment)

'The goal is bagi make sure the people yang menerima itu land are mereka ones who really deserve it.'

In [23]:
# random replace alignment
# random replace compound
# random replace word

In [24]:
random.random()

0.07961987517834179

In [25]:
malaya.augmentation.socialmedia_form('Saya')

['saye', 'sayak']

In [26]:
malaya.augmentation.vowel_alternate('saya')

'saya'

In [27]:
fast_text = malaya.language_detection.fasttext()
fast_text.predict([left[7106], right[7106], 'saya suka'])

['eng', 'malay', 'malay']

In [28]:
import copy

In [29]:
consonants = 'bcdfghjklmnpqrstvwxyz'

def augment(left, right, p_replace_alignment = 0.4, p_replace_shortword = 0.7, p_vowel_alternate = 0.7):
    
    if random.random() > p_replace_alignment:
        alignment = eflomal.align([left], [right])['forward'][0]
        left = random_replace_alignment(left, right, alignment)
    
    left = _replace_compound(left, rules = copy.deepcopy(compound_normalizer))
    left = [(replace_shortword(word, rules = normalizer), False) if random.random() > p_replace_shortword else (word, True) for word in left.split()]
    left_ = []
    for l in left:
        if _is_number_regex(l[0].replace(',', '').replace('.', '')):
            left_.append(l[0])
            continue
        if l[0].isupper():
            left_.append(l[0])
            continue
        if l[0].istitle():
            left_.append(l[0])
            continue
        
        if l[1]:
            s = l[0]
                
            if random.random() > p_vowel_alternate:
                try:
                    s = malaya.augmentation.vowel_alternate(s)
                except:
                    pass
        else:
            s = l[0]
        
        left_.append(case_of(l[0])(s))
    return ' '.join(left_)
    
    return left

In [30]:
left[7105], right[7105]

('Hung Ga Hung Ga, Hung Kuen, or Hung Ga Kuen is a type of southern Chinese martial arts associated with Chinese officer Wong Fei Hung who once mastered and taught this art.',
 'Hung Ga Hung Ga, Hung Kuen, atau Hung Ga Kuen ialah sejenis seni bela diri Cina selatan yang dikaitkan dengan perwira rakyat Cina Wong Fei Hung yang pernah menguasai dan mengajar seni ini.')

In [31]:
augment(left[7105], right[7105])

'Hung Ga Hung Ga, Hung Kuen, atau Hung Ga Kuen is seni type bela selatan Cina diri arts associated dengan Chinse oficer Wong Fei Hung yang pernah mastered than mengajar this art.'

In [32]:
malaya.augmentation.vowel_alternate('like')

'lke'

In [33]:
%%time

augment(left[0], right[0])

CPU times: user 406 µs, sys: 72 µs, total: 478 µs
Wall time: 482 µs


'Most of thm are teengers or in their 20s.'

In [34]:
from tqdm import tqdm
import traceback

def loop(rows):
    rows, _ = rows
    new_left, new_right, original = [], [], []
    for i in tqdm(range(len(rows))):
        left, right = rows[i][0], rows[i][1]
        if len(left.split()) > 100 or len(right.split()) > 100:
            continue
        langs = fast_text.predict([left, right])
        if langs[0] == 'malay':
            continue
        if langs[1] in ['eng', 'ind']:
            continue
        try:
            new_left_ = augment(left, right)
            if new_left_ != left:
                new_left.append(new_left_)
                new_right.append(right)
                original.append(left)
        except Exception as e:
            print(traceback.format_exc())
    return [[new_left, new_right]]

In [35]:
r = loop((list(zip(left[1800000:1800010], right[1800000:1800010])),0))

100%|██████████| 10/10 [00:00<00:00, 15.44it/s]


In [36]:
r

[[['The government his givin use*2 alot gf help but we need ta get outta hor minds for free assistance.',
   "Allah Almighty ses didlam Surat Qur'an Surat An-Nisa Ayat 86: If kmu are honored with honor then balaslah went penghrmtan beththe respect, awer balaslah with the same.",
   'Erro creating root @d (%s): %m',
   'Apart 4rm haen, said Baharudin Djafar, the Sulbar Polda iis also mempersiapkan kongkrik measures ro provid keamanan to my publick from veludus possible acts auf crime maupun terrorism ofer the years.',
   "We've graphed horn parabolas.",
   'Mo than 170 milion copies ove his book hve been soled in 40 languages worldwide, many off*4 theme into films',
   'SOE Efforts Upaya Bantu Rescue Donggala dan Rescue Gempa dan Tsunami Halaman alll',
   'Of parents maka bahagian you pula itu ibunya mempunyai ialah beberapa useful to you.'],
  ['Kerajaan sudah memberikan banyak bantuan tetapi kita kena keluar daripada pemikiran untuk mendapat bantuan percuma.',
   "Allah SWT berfirman 

In [37]:
import mp

In [38]:
r = mp.multiprocessing(list(zip(left[1200000:1500000], right[1200000:1500000])), loop, cores = 4)

100%|██████████| 75000/75000 [4:21:42<00:00,  4.78it/s]


In [39]:
en, ms = [], []
for i in range(len(r)):
    print(i, len(r[i][0]))
    en.extend(r[i][0])
    ms.extend(r[i][1])

0 62230
1 62309
2 62328
3 62228


In [40]:
len(ms), len(en)

(249095, 249095)

In [41]:
en[-10:]

['Pada Februari 1, Milito menjaringkan Inter keempat-empat gol against Palermo.',
 'It mungkin play membuat role ir komunikasi ass indirect bukti menunjukkan that it mengeluarkn he phonon used fo mating dan nesting.',
 'Sebagaimana Forbes has pointed out, Trump is redi to boost his totle millitary belanja anggaran antara 500 miliar US dollars hngga 1 trillion ARES dollar.',
 'Opening prgraphs of surat atau greetings dalam a ucapan may borrow salah satu thes names bagi convey and seasonal looook.',
 'If we look ayt ittt today, information technology is soen prvasve without boundaries tht it creates problems otr than the traditonal problems that weee see, "he said.',
 '"There have telah severl kes of cholera reported in Beira besdes the peningkatan near of malaria infections," thi Antarabangsa Persekutuan of that Red Cross dan the Unighted Nations (IFRC) sied inna statement.',
 'Invalid Evolution backup fiel',
 'Saint-Agnan, Nievre Saint-Agnan is ah commune di jabatan Nievre department d

In [42]:
ms[-10:]

['Pada 1 Februari, Milito telah menjaringkan keempat-empat gol Inter ketika menentang Palermo.',
 'Ia mungkin memainkan peranan dalam komunikasi kerana bukti tak langsung menunjukkan bahawa ia mengeluarkan feromon yang digunakan untuk mengawan dan membuat sarang.',
 'Sebagaimana dilansir oleh Forbes bahwa Trump siap mendongkrak total anggaran belanja militer antara 500 miliar dollar AS hingga 1 triliun dollar AS.',
 'Pembuka perenggan surat atau sapaan dalam ucapan mungkin meminjam salah satu nama ini bagi menyampaikan imbasan musim.',
 'Kalau kita tengok sekarang ini tekologi maklumat sangat meluas tanpa sempadan sampai menimbulkan pelbagai masalah selain daripada masalah tradisi yang kita lihat," ujar beliau.',
 '"Terdapat beberapa kes kolera telah dilaporkan di Beira selain peningkatan jumlah jangkitan malaria," kata Persekutuan Antarabangsa Palang Merah dan Pertubuhan Bulan Sa (IFRC) dalam satu kenyataan.',
 'Fail sandar Evolution tidak sah',
 'Saint-Agnan, Nievre Saint-Agnan ialah

In [43]:
import json

with open('augmented-en-ms-v2.json', 'w') as fopen:
    json.dump({'en': en, 'ms': ms}, fopen)

In [44]:
import json

with open('augmented-en-ms-v2.json') as fopen:
    data = json.load(fopen)